# Exploring trends that lead to outperforming weekly/daily price action

This notebook focuses on analyzing the volume/price action that lead to a significant price outperformance for lower cap coins. The pycoingecko library is used to retrieve token information from Coingecko.

The goal is to determine if there is any trend that tends to lead to a token having price action that outperforms the rest of the market. 

1. Extract price and volume data for tokens over the past month
2. Calculate daily price increases, over the previous day and week
3. Identify which tokens had the highest price increase for each day
4. Create a dataframe with each days top mover and the preceding price/volume/etc data


In [2]:
#pip install pandas
#pip install pycoingecko
#pip install matplotlib

## 0. Setup

### Dependencies

In [3]:
from pycoingecko import CoinGeckoAPI
from datetime import datetime
import pandas as pd
import time
from io import StringIO
import matplotlib.pyplot as plt
import requests

### Define functions

In [4]:
cg = CoinGeckoAPI()

# Get response for list of coins -> transforms into a single dataframe



# Get response from 'get_coin_market_chart_by_id' call = returns dict of daily mcap, vol + price per coin

def get_market_data_response(coin_id: str, days: int):
    coin_market_data = cg.get_coin_market_chart_by_id(id=coin_id, 
                                                      vs_currency='usd', 
                                                      days=days, 
                                                      interval='daily')
    coin_market_data['id'] = coin_id
    return coin_market_data

# test=get_market_data_reponse('tron')
# test


#calc various statistics

def calc_columns(id, df_coin):
    df_coin["1_day_return"] = df_coin["price"].pct_change(1) * 100
    df_coin["7_day_return"] = df_coin["price"].pct_change(7) * 100
  
    df_coin["1_day_vol_dif"] = df_coin["volume"].pct_change(1) * 100
    df_coin["7_day_vol_dif"] = df_coin["volume"].pct_change(7) *100
    
    #prefix the columns with coin id
    col_prefix = f"{id}_"
    df_coin=df_coin.add_prefix(col_prefix)
    
    
    return df_coin


#convert market-data json response for each coin into a dataframe

def response_to_df(coin_market_data):
    coin_id = coin_market_data['id']
    
    df_coin = pd.DataFrame(coin_market_data['prices'], columns=['date', 'price'])
    
    #clean up date and set as index
    df_coin['date']=pd.to_datetime(df_coin['date'], unit='ms')
    df_coin['date'] = pd.to_datetime(df_coin['date']).dt.date
    df_coin.set_index('date', inplace=True)
    
    # attach volume data
    coin_volume = coin_market_data['total_volumes']
    volume_list = []
    for volume in coin_volume:
        volume_list.append(volume[1])

    df_coin['volume'] = volume_list
    
    #drop any duplicate indices
    df_coin = df_coin[~df_coin.index.duplicated(keep='first')]
    
    # calc various columns
    df_coin = calc_columns(coin_id, df_coin)
    
    
    return df_coin
    
# test2 = response_to_df(test)
# test2


## 1. Get price and volume data for tokens

In [5]:
# Adjust what range of tokens to explore by changing the page variables (indexed by coingeck mcap rank)
# If "per_page" is 5, and "from_page" is 7, then it will start with #35 mcap token

def get_coins(per_page: int, from_page: int, to_page: int):
    """
    Purpose: Get JSON from coingecko api call, and convert into a dataframe
    """
    coins_list_df = pd.DataFrame()
    
    page = from_page #wherever I want to start
    
    while page < to_page: #however many I want to get
        coins_market_response = cg.get_coins_markets(vs_currency = 'usd',
                                                page = page,
                                                per_page = per_page, #to test use 3, for prod use 250
                                                price_change_percentage = '24h,7d,30d')
        output_df = pd.DataFrame(coins_market_response)

        coins_list_df = pd.concat([coins_list_df, output_df])
        
        
        page+=1
        
        
    return coins_list_df

In [6]:
coins_list_df = get_coins(per_page = 5, from_page = 6, to_page = 7)
coins_list_df

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,price_change_percentage_24h_in_currency,price_change_percentage_30d_in_currency,price_change_percentage_7d_in_currency
0,near,near,NEAR Protocol,https://assets.coingecko.com/coins/images/1036...,3.490000,2523382426,26,3.492295e+09,181714146,3.790000,...,-82.91284,2022-01-16T22:09:45.873Z,0.526762,562.97347,2020-11-04T16:09:15.137Z,None,2022-06-28T17:30:24.614Z,-6.520246,-30.377587,1.893442
1,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.140000,2343136409,27,5.132946e+09,108341491,5.450000,...,-88.56453,2021-05-03T05:25:04.822Z,1.030000,398.61556,2020-09-17T01:20:38.214Z,None,2022-06-28T17:30:18.610Z,-5.423213,2.878004,19.162203
2,cosmos,atom,Cosmos Hub,https://assets.coingecko.com/coins/images/1481...,7.630000,2234504003,28,NaN,320048130,8.110000,...,-82.81925,2022-01-17T00:34:41.497Z,1.160000,558.28580,2020-03-13T02:27:44.591Z,"{'times': 75.33202025973254, 'currency': 'usd'...",2022-06-28T17:30:35.694Z,-5.137156,-19.163050,11.545576
3,algorand,algo,Algorand,https://assets.coingecko.com/coins/images/4380...,0.319207,2203026320,29,3.193149e+09,82538713,0.336169,...,-90.61343,2019-06-20T14:51:19.480Z,0.105336,217.33874,2020-03-13T02:20:48.438Z,"{'times': -0.866997122829602, 'currency': 'usd...",2022-06-28T17:30:36.867Z,-3.650983,-13.233348,-0.682401
4,ethereum-classic,etc,Ethereum Classic,https://assets.coingecko.com/coins/images/453/...,16.010000,2166229522,30,3.368136e+09,196990188,16.790000,...,-90.45503,2021-05-06T18:34:22.133Z,0.615038,2493.04949,2016-07-25T00:00:00.000Z,"{'times': 34.579909560786845, 'currency': 'usd...",2022-06-28T17:30:29.638Z,-3.733088,-29.848026,-2.018572


### Calculate statistics on all tokens and return a single df with price, volume & % change data

In [7]:
# get full dataframe - combine dataframes
def get_complete_dataframe(coins_list_df):
    
    num_requests = 0
    complete_df = pd.DataFrame()
    
    for i, row in coins_list_df.iterrows():
        
        coin_id = row['id']
        print(coin_id)
        
        
        try:
            response=get_market_data_response(coin_id = coin_id, days=37)
        
        except requests.exceptions.HTTPError  as e:
            print("Reached request limit: waiting 70 seconds")
            num_requests = 0
            time.sleep(70)
        
        
#         if num_requests == 45:
#             print("Waiting 65 seconds to not exceed request limit...")
#             num_requests = 0
#             time.sleep(75)
#         response=get_market_data_reponse(coin_id)
#         num_requests += 1
        
        coin_df = response_to_df(response)
        
        
        complete_df = pd.concat([complete_df, coin_df], axis = 1)
    
    return(complete_df)

In [8]:
all_coins_df = get_complete_dataframe(coins_list_df)
all_coins_df

near
uniswap
cosmos
algorand
ethereum-classic


,near_price,near_volume,near_1_day_return,near_7_day_return,near_1_day_vol_dif,near_7_day_vol_dif,uniswap_price,uniswap_volume,uniswap_1_day_return,uniswap_7_day_return,...,algorand_1_day_return,algorand_7_day_return,algorand_1_day_vol_dif,algorand_7_day_vol_dif,ethereum-classic_price,ethereum-classic_volume,ethereum-classic_1_day_return,ethereum-classic_7_day_return,ethereum-classic_1_day_vol_dif,ethereum-classic_7_day_vol_dif
date,,,,,,,,,,,,,,,,,,,,,
2022-05-23,6.260143,2.903980e+08,NaN,NaN,NaN,NaN,5.512928,1.090627e+08,NaN,NaN,...,NaN,NaN,NaN,NaN,21.144274,1.969587e+08,NaN,NaN,NaN,NaN
2022-05-24,5.824683,3.307652e+08,-6.956061,NaN,13.900643,NaN,5.624137,1.842489e+08,2.017254,NaN,...,-6.452062,NaN,22.379035,NaN,21.966883,8.498067e+08,3.890457,NaN,331.464464,NaN
2022-05-25,5.974107,3.591201e+08,2.565352,NaN,8.572503,NaN,5.597278,1.656751e+08,-0.477579,NaN,...,0.472127,NaN,-18.294442,NaN,24.378902,7.508639e+08,10.980254,NaN,-11.642978,NaN
2022-05-26,5.742704,3.244003e+08,-3.873434,NaN,-9.668002,NaN,5.617416,1.410522e+08,0.359788,NaN,...,-3.120994,NaN,1.141976,NaN,23.758426,6.707109e+08,-2.545137,NaN,-10.674767,NaN
2022-05-27,5.254139,4.906144e+08,-8.507570,NaN,51.237339,NaN,4.975035,1.753480e+08,-11.435521,NaN,...,-5.715561,NaN,30.591323,NaN,22.141108,4.695886e+08,-6.807344,NaN,-29.986435,NaN
2022-05-28,4.790383,4.741700e+08,-8.826499,NaN,-3.351806,NaN,4.674747,1.813743e+08,-6.035906,NaN,...,-5.810931,NaN,-9.496511,NaN,22.539989,6.269737e+08,1.801538,NaN,33.515521,NaN
2022-05-29,5.014805,2.338532e+08,4.684857,NaN,-50.681559,NaN,4.994721,1.022104e+08,6.844733,NaN,...,2.337812,NaN,-27.091380,NaN,22.823249,2.502491e+08,1.256700,NaN,-60.086190,NaN
2022-05-30,5.178482,2.514232e+08,3.263869,-17.278531,7.513242,-13.421170,4.947217,9.244452e+07,-0.951079,-10.261528,...,2.965310,-14.792648,-7.222229,-19.147675,23.740981,2.370817e+08,4.021042,12.280900,-5.261722,20.371280
2022-05-31,5.759712,3.990562e+08,11.223950,-1.115444,58.718911,20.646356,5.646552,1.504855e+08,14.135932,0.398548,...,10.752768,0.878227,43.530575,-5.173457,24.870620,3.473554e+08,4.758183,13.218707,46.512947,-59.125368


## 2. Query for the top 5 tokens with the largest 7-day return for each day, and get data leading up to the day they pumped

In [9]:
def top_5_returns(all_coins_df):

    #filter to only 7_day_return cols and remove first 7 rows (nan values)
    weekly_return_df = all_coins_df[[col for col in all_coins_df.columns if '7_day_return' in col]]
    weekly_return_df = weekly_return_df.iloc[7:]
    #weekly_return_df = weekly_return_df.dropna(axis=1)

    # get top 5 weekly movers by for each date
    top_n = 5
    top_movers = pd.DataFrame({n: weekly_return_df.T[col].nlargest(top_n).index.tolist() 
                      for n, col in enumerate(weekly_return_df.T)}).T


    #add the date and set as index
    temp_date_df = weekly_return_df.reset_index()
    final_top = pd.concat([top_movers, temp_date_df["date"]], axis = 1)
    final_top['date'] = pd.to_datetime(final_top['date']).dt.date
    final_top.set_index('date', inplace=True)

    final_top = final_top.replace({'_7_day_return': ''}, regex=True)
    return final_top

In [10]:
top_coins_weekly_returns = top_5_returns(all_coins_df)
top_coins_weekly_returns

,0,1,2,3,4
date,,,,,
2022-05-30,ethereum-classic,uniswap,algorand,cosmos,near
2022-05-31,ethereum-classic,algorand,uniswap,near,cosmos
2022-06-01,uniswap,near,algorand,ethereum-classic,cosmos
2022-06-02,algorand,ethereum-classic,near,uniswap,cosmos
2022-06-03,uniswap,near,algorand,ethereum-classic,cosmos
2022-06-04,near,uniswap,algorand,cosmos,ethereum-classic
2022-06-05,near,algorand,uniswap,cosmos,ethereum-classic
2022-06-06,algorand,uniswap,near,cosmos,ethereum-classic
2022-06-07,algorand,near,uniswap,ethereum-classic,cosmos


### Get price + volume data leading up to day it had a large increase in price

In [13]:

def pa_pump_df(top_coins_weekly_returns):
    """
    Create dateaframe with all price action leading up to the day it had the high weekly return
    """
    
    weekly_dict = top_coins_weekly_returns.to_dict('index')

    num_requests = 0

    weekly_movers_df = pd.DataFrame()

    for date, values in weekly_dict.items():
        print(date)
        for i, coin in values.items():
            #print(coin)
            try:
                response=get_market_data_response(coin_id = coin, days = 60)

            except requests.exceptions.HTTPError  as e:
                print("Reached request limit: waiting 70 seconds")
                num_requests = 0
                time.sleep(70)
                
            num_requests += 1
            
            df_coin = response_to_df(response)
            
            # reverse df so it is in descending order
            df_coin = df_coin.loc[::-1]
 
            #---------------------------
            #reset index
            df_coin = df_coin.reset_index()
            
            #Filter to only include prices leading up to the date where it had the highest weekly return
            df_coin = df_coin[(df_coin['date'] <= date)]
            
            #!!! Reset the index again and remove it
            df_coin = df_coin.reset_index()
            df_coin = df_coin.drop(['date','index'], axis = 1)
            
            #---------------------------
            
            #rename columns
            col_prefix = f"{date}_"
            df_coin=df_coin.add_prefix(col_prefix)
            

            #concatenate to the final db 
            weekly_movers_df = pd.concat([weekly_movers_df, df_coin], axis = 1)

    return weekly_movers_df

Output:
- Each column represents the price action leading up to the day it had a large weekly increase. Index 0 is the data for that specific day, and each row below represents the data a day before. (ie index 1 is one day before the pump, index 2 is two days before the pump, etc)  
- the naming convention for each column: {date of large increase}_{token name}_{metric_name}

In [14]:
df_price_action_before_pump = pa_pump_df(top_coins_weekly_returns)
df_price_action_before_pump

2022-05-30
2022-05-31
2022-06-01
2022-06-02
2022-06-03
2022-06-04
2022-06-05
2022-06-06
2022-06-07
2022-06-08
2022-06-09
2022-06-10
2022-06-11
2022-06-12
2022-06-13
2022-06-14
Reached request limit: waiting 70 seconds
2022-06-15
2022-06-16
2022-06-17
2022-06-18
2022-06-19
2022-06-20
2022-06-21
2022-06-22
Reached request limit: waiting 70 seconds
2022-06-23
2022-06-24
2022-06-25
2022-06-26
2022-06-27
2022-06-28


,2022-05-30_ethereum-classic_price,2022-05-30_ethereum-classic_volume,2022-05-30_ethereum-classic_1_day_return,2022-05-30_ethereum-classic_7_day_return,2022-05-30_ethereum-classic_1_day_vol_dif,2022-05-30_ethereum-classic_7_day_vol_dif,2022-05-30_uniswap_price,2022-05-30_uniswap_volume,2022-05-30_uniswap_1_day_return,2022-05-30_uniswap_7_day_return,...,2022-06-28_algorand_1_day_return,2022-06-28_algorand_7_day_return,2022-06-28_algorand_1_day_vol_dif,2022-06-28_algorand_7_day_vol_dif,2022-06-28_ethereum-classic_price,2022-06-28_ethereum-classic_volume,2022-06-28_ethereum-classic_1_day_return,2022-06-28_ethereum-classic_7_day_return,2022-06-28_ethereum-classic_1_day_vol_dif,2022-06-28_ethereum-classic_7_day_vol_dif
0,23.740981,2.370817e+08,4.021042,12.280900,-5.261722,20.371280,4.947217,9.244452e+07,-0.951079,-10.261528,...,-1.050644,2.768133,-2.544607,-20.823297,16.588060,2.429826e+08,-0.852571,1.513083,15.078607,4.515421
1,22.823249,2.502491e+08,1.256700,11.871394,-60.086190,91.640522,4.994721,1.022104e+08,6.844733,-4.944798,...,-4.934474,6.169604,-4.231587,-25.662570,16.730701,2.111449e+08,1.221626,9.009222,52.289991,-7.767278
2,22.539989,6.269737e+08,1.801538,11.551724,33.515521,203.885792,4.674747,1.813743e+08,-6.035906,-8.577571,...,-0.374046,18.291577,-23.718116,-24.005834,16.528781,1.386466e+08,0.521458,20.268858,-23.652524,-39.919225
3,22.141108,4.695886e+08,-6.807344,5.561651,-29.986435,114.619195,4.975035,1.753480e+08,-11.435521,-4.263332,...,4.871527,13.133781,28.131289,30.563146,16.443038,1.815995e+08,3.165479,14.093091,27.394944,11.798737
4,23.758426,6.707109e+08,-2.545137,19.690510,-10.674767,179.775524,5.617416,1.410522e+08,0.359788,14.886046,...,6.666939,11.343927,-6.109004,-33.638341,15.938508,1.425484e+08,4.750991,14.947945,-20.548139,-35.315438
5,24.378902,7.508639e+08,10.980254,11.570830,-11.642978,227.301309,5.597278,1.656751e+08,-0.477579,1.981495,...,-3.256129,-7.323035,-21.615761,-62.103028,15.215615,1.794148e+08,-5.524309,-3.799230,-15.742033,-54.983454
6,21.966883,8.498067e+08,3.890457,5.083454,331.464464,173.598582,5.624137,1.842489e+08,2.017254,11.499946,...,1.330237,5.159941,17.934248,-29.798058,16.105323,2.129351e+08,-1.441099,9.921026,-8.409066,-35.217902
7,21.144274,1.969587e+08,3.641661,-4.488944,50.830772,-11.129441,5.512928,1.090627e+08,4.917267,1.182208,...,2.224431,6.317216,-8.501072,-54.587431,16.340810,2.324850e+08,6.468880,14.199825,1.554518,-47.274380
8,20.401327,1.305826e+08,0.967361,-4.171050,-36.708377,-56.408458,5.254548,1.034380e+08,2.761394,1.117040,...,5.919685,-3.916551,-2.097225,10.543618,15.347969,2.289263e+08,11.676876,-8.690471,-0.797672,-1.093428
9,20.205863,2.063189e+08,-3.664972,-2.052402,-5.704727,-47.307094,5.113348,1.460656e+08,-1.601725,0.636086,...,-4.717976,-14.442555,31.057466,17.452351,13.743193,2.307670e+08,-4.640287,-24.464877,42.067931,17.146554


# 3. Analysis

Ideas:
- Standard deviation across columns
- Correlation between price and volume

Todo:
- add calculated values to final dataframes
- show filtering to specific columns ie price, 7day price, etc
- For analysis filter to each specific column and explore those trends

In [23]:
#Average 7-day price action leading to gain:

#filter to "7_day_return" columns
price_seven_df = df_price_action_before_pump[[col for col in df_price_action_before_pump.columns if '7_day_return' in col]]

print(price_seven_df)

#print(price_seven_df.mean(axis=1))

    2022-05-30_ethereum-classic_7_day_return  2022-05-30_uniswap_7_day_return  \
0                                  12.280900                       -10.261528   
1                                  11.871394                        -4.944798   
2                                  11.551724                        -8.577571   
3                                   5.561651                        -4.263332   
4                                  19.690510                        14.886046   
5                                  11.570830                         1.981495   
6                                   5.083454                        11.499946   
7                                  -4.488944                         1.182208   
8                                  -4.171050                         1.117040   
9                                  -2.052402                         0.636086   
10                                 10.220126                        10.980562   
11                          

# Plotting

In [22]:
# for i, col in enumerate(price_seven_df.columns):
#     price_seven_df[col].plot()

# plt.title('7 Day Price Action Comparison')

# plt.xticks(rotation=70)
# plt.legend(price_seven_df.columns)